In [1]:
##imports 

import cv2 
import pytesseract
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix


In [4]:
##Needed CSV's
trainingData = pd.read_csv("train-data.csv", index_col=0) 
testingData  = pd.read_csv("test.csv", index_col=0) 

In [6]:
##batch size (started with a small no. to train the algorithm on)
batch_size = 320000

In [19]:
##vectorizing the data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(trainingData[:batch_size]['OCR text'])

In [12]:
y = trainingData[:batch_size]["Label"]

In [13]:

print(X)

  (0, 1023804)	0.17356085231708732
  (0, 3902906)	0.11006352215199676
  (0, 1994521)	0.17368667202763452
  (0, 1105205)	0.10576162854433116
  (0, 486789)	0.2621773100694327
  (0, 613354)	0.2228816039914777
  (0, 615186)	0.20749094914032148
  (0, 3269928)	0.12909278246115696
  (0, 1565801)	0.10914941928698993
  (0, 2774797)	0.0990143902998192
  (0, 3522610)	0.10883672516232283
  (0, 2869639)	0.09994235132014762
  (0, 3269883)	0.22967632373744243
  (0, 1154145)	0.1358293341451583
  (0, 640366)	0.16160764306247738
  (0, 3387834)	0.07002938197734061
  (0, 1271309)	0.08842233560437013
  (0, 1111183)	0.11582977767520121
  (0, 3280571)	0.24496235065522273
  (0, 346899)	0.2119889565398721
  (0, 3387681)	0.08155898259578934
  (0, 2918596)	0.21840229149025575
  (0, 785441)	0.2460772720436303
  (0, 362384)	0.22574311148663473
  (0, 2169322)	0.2748617352756527
  :	:
  (319999, 1347754)	0.3541882361429565
  (319999, 2597553)	0.3541882361429565
  (319999, 378591)	0.2831840859748654
  (319999, 230298

In [9]:
print(vectorizer.get_feature_names())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
print(X.shape)


(320000, 3963685)


In [24]:
##Creating pipelines for Naive Bayes

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
mnb_pipeline = Pipeline([
    ('nb', MultinomialNB())
])

In [34]:
##Listing the testing tuning parameters

import numpy as np
from sklearn.model_selection import GridSearchCV
grid_params = {
  'nb__alpha': np.linspace(0.01, 0.02, 1),
  'nb__fit_prior': [True, False],
}
clf = GridSearchCV(mnb_pipeline, grid_params)
clf.fit(X, y)
print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)


Best Score:  0.670021875
Best Params:  {'mnb__alpha': 0.01, 'mnb__fit_prior': False}


In [42]:
##Fiting the data and training it - modify tuning parameters here based on the GridsearchCV

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.01)

clf.fit(X, y)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [43]:
##Testing data based on the fitted model

from tqdm import tqdm

#Prediction and accuracy
from sklearn.metrics import accuracy_score
documents1 = testingData[:]["OCR text"]
y_true = testingData[:]['Label']
input = vectorizer.transform(documents1)
prediction1= clf.predict(input)

dataframe = pd.DataFrame(list(y_true),prediction1)
index = 0
count = 0
for i in tqdm(list(y_true)):
    if i == prediction1[index]:
        count += 1
    index+=1
    
    
print("Accuracy:",accuracy_score(list(y_true),prediction1))
print(dataframe)

100%|██████████| 40000/40000 [00:00<00:00, 718802.77it/s]


0.680225
     0
11  11
12   0
5    0
12   4
14  14
..  ..
14  10
11  11
9   15
9    9
15  15

[40000 rows x 1 columns]
